In [ ]:
# !pip install requests
# !pip install beautifulsoup4
# !pip install langchain-community
# !pip install langchain-text-splitters
# !pip install langchain-chroma
# !pip install -U langchain-ollama

In [ ]:
# !pip install requests beautifulsoup4 langchain-community langchain-text-splitters langchain-chroma 

In [2]:
import os

# ตั้งค่าให้ Ollama ใช้เซิร์ฟเวอร์ที่รันอยู่บนเครื่องอื่น
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"

# ตรวจสอบว่าตัวแปรถูกตั้งค่าแล้วหรือไม่
print(os.getenv("USER_AGENT"))  # ควรแสดง http://server-ip:11434

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36


In [3]:
import bs4
import requests
from langchain_community.document_loaders import WebBaseLoader

# Monkey Patch requests.get เพื่อใส่ User-Agent
original_get = requests.get

def patched_get(url, *args, **kwargs):
    headers = kwargs.pop("headers", {})
    headers["User-Agent"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
    return original_get(url, headers=headers, *args, **kwargs)

requests.get = patched_get  # ใช้ patched version ของ requests.get

# ใช้ WebBaseLoader ตามปกติ
bs4_strainer = bs4.SoupStrainer(class_="EntryReaderInner")
loader = WebBaseLoader(
    web_paths=("https://www.sanook.com/news/9735274/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

# คืนค่า requests.get กลับเป็นปกติ (ถ้าต้องการ)
requests.get = original_get

print(docs)

[Document(metadata={'source': 'https://www.sanook.com/news/9735274/'}, page_content='เด็กสาววัย 16 ไปโรงพยาบาลเพราะท้องอืด ช็อก หมอบอกมีชีวิตอยู่ได้อีกแค่ 1 สัปดาห์ ต้องผ่าตัดด่วน\nอเล็กซ์ อาร์เคล อายุ 16 ปี จากสติริง สกอตแลนด์ ไปพบแพทย์เพื่อตรวจสอบอาการท้องอืดและปวดท้องที่เธอประสบอยู่ในช่วงหลัง ซึ่งไม่เคยคาดคิดว่าเธอจะเหลือเวลาอีกแค่ 1 สัปดาห์ในการมีชีวิตอยู่จากการมีเนื้องอกที่หายากมากในโลก\nอเล็กซ์สังเกตเห็นอาการไม่สบายตัวครั้งแรกในระหว่างทริปท่องเที่ยวกับครอบครัว และเมื่อกลับบ้าน เธอก็รีบไปนัดหมายตรวจสุขภาพกับแพทย์ เธอได้รับการตรวจเลือดและเพียงแค่สัปดาห์เดียวหลังจากนั้นก็ถูกนำตัวไปโรงพยาบาลเพื่อทำการอัลตราซาวด์ ขณะนั้นเธอสูญเสียการมองเห็นในสายตาซ้ายและได้รับการแจ้งว่าเนื้องอกของเธอเติบโตขึ้นอีก 13 เซนติเมตร\nอเล็กซ์ได้รับการแจ้งว่าเธอเป็นโรคเนื้องอกต่อมใต้สมองที่หายาก ซึ่งในโลกนี้มีการบันทึกกรณีเช่นนี้เพียง 20 ราย และเธอเหลือเวลาแค่ 1 สัปดาห์เท่านั้น "เมื่ออายุ 16 ปี ชีวิตของฉันเหมือนจะผ่านไปต่อหน้าต่อตา สิ่งที่ฉันเคยห่วงใยมันไม่สำคัญสำหรับฉันอีกต่อจากวันนั้น" นักเรียนหญิงเผยว่าเธอไ

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200, chunk_overlap=100, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)
print(all_splits)

[Document(metadata={'source': 'https://www.sanook.com/news/9735274/', 'start_index': 0}, page_content='เด็กสาววัย 16 ไปโรงพยาบาลเพราะท้องอืด ช็อก หมอบอกมีชีวิตอยู่ได้อีกแค่ 1 สัปดาห์ ต้องผ่าตัดด่วน\nอเล็กซ์ อาร์เคล อายุ 16 ปี จากสติริง สกอตแลนด์ ไปพบแพทย์เพื่อตรวจสอบอาการท้องอืดและปวดท้องที่เธอประสบอยู่ในช่วงหลัง ซึ่งไม่เคยคาดคิดว่าเธอจะเหลือเวลาอีกแค่ 1 สัปดาห์ในการมีชีวิตอยู่จากการมีเนื้องอกที่หายากมากในโลก\nอเล็กซ์สังเกตเห็นอาการไม่สบายตัวครั้งแรกในระหว่างทริปท่องเที่ยวกับครอบครัว และเมื่อกลับบ้าน เธอก็รีบไปนัดหมายตรวจสุขภาพกับแพทย์ เธอได้รับการตรวจเลือดและเพียงแค่สัปดาห์เดียวหลังจากนั้นก็ถูกนำตัวไปโรงพยาบาลเพื่อทำการอัลตราซาวด์ ขณะนั้นเธอสูญเสียการมองเห็นในสายตาซ้ายและได้รับการแจ้งว่าเนื้องอกของเธอเติบโตขึ้นอีก 13 เซนติเมตร\nอเล็กซ์ได้รับการแจ้งว่าเธอเป็นโรคเนื้องอกต่อมใต้สมองที่หายาก ซึ่งในโลกนี้มีการบันทึกกรณีเช่นนี้เพียง 20 ราย และเธอเหลือเวลาแค่ 1 สัปดาห์เท่านั้น "เมื่ออายุ 16 ปี ชีวิตของฉันเหมือนจะผ่านไปต่อหน้าต่อตา สิ่งที่ฉันเคยห่วงใยมันไม่สำคัญสำหรับฉันอีกต่อจากวันนั้น" นักเ

In [5]:
import os

# ตั้งค่าให้ Ollama ใช้เซิร์ฟเวอร์ที่รันอยู่บนเครื่องอื่น
os.environ["OLLAMA_BASE_URL"] = "http://ollama:11434"

# ตรวจสอบว่าตัวแปรถูกตั้งค่าแล้วหรือไม่
print(os.getenv("OLLAMA_BASE_URL"))  # ควรแสดง http://server-ip:11434

http://ollama:11434


In [6]:
from langchain_ollama import OllamaEmbeddings

# ใช้ OllamaEmbeddings จากเซิร์ฟเวอร์ที่กำหนด
local_embeddings = OllamaEmbeddings(
    model="all-minilm",
    base_url="http://ollama:11434"
)

# ทดสอบสร้าง embedding
text = "สวัสดี! วันนี้อากาศดีจัง"
embedding = local_embeddings.embed_query(text)

print(embedding)  # จะได้ค่าเวกเตอร์ของข้อความ

[0.013249871, -0.040880237, 0.094288655, -0.019420143, 0.020536907, 0.0302462, 0.09664201, -0.02962447, 0.03134943, -0.021218855, 0.046145417, -0.08314556, 0.12087323, 0.010088535, -0.05782056, -0.06926961, -0.016928507, -0.014842418, -0.05266153, 0.035300795, -0.011006773, 0.038736004, 0.038296945, 0.038405865, -0.05490089, 0.05476305, -0.052712232, 0.007348415, 0.04534394, -0.05935963, 0.06652948, 0.012969663, -0.06612183, -0.07018619, 0.015783165, 0.008019178, 0.009855329, -0.038495086, -0.05829462, 0.027262945, -0.08911476, -0.08241787, 0.06943403, -0.08724207, 0.055218972, 0.024746163, -0.0683722, -0.00558454, 0.06059762, 0.026932606, -0.0071486505, 0.047604654, 0.010693587, 0.0046240343, 0.013059228, -0.012057707, -0.031619485, -0.090213485, -0.027809149, -0.052080095, -0.031833433, 0.038915478, 0.030485243, -0.0030363544, 0.009985302, -0.016163051, 0.006033955, -0.02187463, 0.014055816, 0.07612431, 0.036380794, -0.0036411542, 0.05290088, -0.010547627, -0.0100627765, 0.015535286,

In [7]:
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

local_embeddings = OllamaEmbeddings(
    model="bge-m3",
    base_url="http://ollama:11434"
)
vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

question = "what are the oversold and overbought periods?"
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retrieved_docs = retriever.invoke(question)

print(retrieved_docs)

Number of requested results 3 is greater than number of elements in index 2, updating n_results = 2


[Document(id='f5e080bf-5721-49ff-bcf9-5d5c82c84af1', metadata={'source': 'https://www.sanook.com/news/9735274/', 'start_index': 989}, page_content='"ฉันถูกนำตัวไปโรงพยาบาลเพื่อผ่าตัดฉุกเฉินในสองวันหลังจากนั้น ฉันไม่รู้ว่าจะอธิบายความรู้สึกของตัวเองในตอนนั้นอย่างไร ในขณะนั้น ฉันไม่รู้ว่ามันจะเปลี่ยนชีวิตของฉันอย่างไร ฉันรู้สึกตกใจ กลัว วิตกกังวลเกี่ยวกับการผ่าตัด ฉันสงสัยว่าฉันจะตายหรือไม่ หรือฉันจะมีแผลเป็นที่ใบหน้าไหม?"\xa0\nโชคร้ายที่อเล็กซ์ประสบปัญหาภายหลังการผ่าตัดในปี 2021 และต้องนอนรักษาตัวในโรงพยาบาล 2 สัปดาห์ ในระหว่างนี้ เธอได้รับการตรวจสอบอย่างต่อเนื่องทั้งฮอร์โมนและเลือด รวมถึงติดตามสภาพสุขภาพทุกชั่วโมง\nหลังจากผ่านพ้นวิกฤต อเล็กซ์ได้รับการวินิจฉัยว่าเป็นโรค PTSD (ภาวะเครียดหลังเหตุการณ์สะเทือนใจ) เธอเกือบจะพลาดปีการศึกษาทั้งปี รวมถึงการสอบ เนื่องจากกระบวนการฟื้นฟูใช้เวลานาน 13 เดือนต่อมา อเล็กซ์ได้รับการแจ้งว่าเธอต้องผ่าตัดด่วนอีกครั้งหลังจากเนื้องอกได้เติบโตและกดทับเส้นประสาทตาของเธอ\nในเดือนกุมภาพันธ์ 2023 เธอได้รับการรักษาด้วยโปรตอนบีมเทอราพี ซึ่งเป็นวิธีการรักษาด้วยรังส